In [32]:
import pandas as pd
import numpy as np

import ast
import pickle

import snap

In [2]:
path = 'output/'
cluster_data = pd.read_csv(path+'1_step_user_groups.csv', sep='\t')

In [3]:
cluster_tags = pd.read_csv(path+'1_step_tags_clusters.txt', sep='\t')
cluster_tags.head()

,n_cluster,dim,hashtags
0,-1,1618,/
1,0,6828,/
2,1,16,"interiorinspo,decora,interiordecor,interiorsty..."
3,2,35,"disney,instacat,gato,princess,ladyboss,poetryc..."
4,3,14,"europa,fashionstylist,voyage,argentina,aloha,e..."


In [4]:
# filter clusters that are not relevant
cluster_data = cluster_data[cluster_data['num_users'] > 1]
cluster_data.head()

,id_cluster,num_users,list_users
0,1,17,"('citychic4ever__', 0.9411764705882353), ('raw..."
1,2,16,"('len_cap', 1.0), ('crown_rising', 0.823529411..."
2,3,15,"('rimaglio', 1.0), ('pollyonvoyage', 0.8235294..."
4,5,2,"('mynnorthamerica', 0.8823529411764706), ('gra..."
5,6,9,"('thankstoyou_pgh', 0.9411764705882353), ('njc..."


In [48]:
def filterUsers(userlist):
    userlist = list(ast.literal_eval(userlist))
    result = []
    for userdata in userlist:
        if userdata[1] <= 0.5:
            result.append(userdata)
    return result

In [6]:
def getCompleteUserlist(userlist):
    userlist = list(ast.literal_eval(userlist))
    result = []
    for userdata in userlist:
        result.append(userdata[0])
    return result

In [49]:
# keep only relevant users (== consumers)
cluster_data['relevant_users'] = cluster_data.apply(lambda x: filterUsers(x['list_users']), axis=1)
cluster_data['all_users'] = cluster_data.apply(lambda x: getCompleteUserlist(x['list_users']), axis=1)
cluster_data.head()

,id_cluster,num_users,list_users,relevant_users,all_users
0,1,17,"('citychic4ever__', 0.9411764705882353), ('raw...","[(rawcontent, 0.5), (magalyopice, 0.375), (lib...","[citychic4ever__, rawcontent, magalyopice, lib..."
1,2,16,"('len_cap', 1.0), ('crown_rising', 0.823529411...","[(ashcebulka, 0.352941176471), (aurotempus, 0....","[len_cap, crown_rising, murrymckee, ashcebulka..."
2,3,15,"('rimaglio', 1.0), ('pollyonvoyage', 0.8235294...","[(melrosexclusive, 0.0588235294118), (manak419...","[rimaglio, pollyonvoyage, melrosexclusive, man..."
4,5,2,"('mynnorthamerica', 0.8823529411764706), ('gra...","[(grantbriskinphl, 0.1)]","[mynnorthamerica, grantbriskinphl]"
5,6,9,"('thankstoyou_pgh', 0.9411764705882353), ('njc...","[(njcubangirl, 0.117647058824), (wandpdesign, ...","[thankstoyou_pgh, njcubangirl, wandpdesign, if..."


In [52]:
with open('output/1_step_consumers.csv', 'w') as outfile:
    outfile.write('n_cluster\tn_users\tconsumers_list\n')
    for c in cluster_data['id_cluster']:
        cluster = cluster_data[cluster_data['id_cluster'] == c]['relevant_users'].values[0]
        dim = len(cluster)
    
        outfile.write('{}\t{}\t{}\n'.format(c, dim, str(cluster).strip('[]')))

In [8]:
# read source network
network = snap.LoadEdgeListNet('data/followers_network.csv', '\t')

V = network.GetNodes()

In [12]:
# use all the users to expand the clusters
temp_result = []
for c in cluster_data['id_cluster']:
    c_users = cluster_data[cluster_data['id_cluster'] == c]['all_users'].values[0]
    
    it = network.BegNI()
    tag_ids = snap.TIntV()
    for i in range(V):
        nid = it.GetId()
        type = network.GetStrAttrDatN(nid, 'type')
        username = network.GetStrAttrDatN(nid, 'content')

        if type == 'user' and username in c_users:
            # add all tags of user
            tags = snap.TIntV()
            snap.GetNodesAtHop(network, nid, 2, tags, True)
            tag_ids.AddV(tags)

        it.Next()
        
    tag_ids.Merge() 
    
    alltags = set()
    for tid in tag_ids:
        tagname = network.GetStrAttrDatN(tid, 'content')
        alltags.add(tagname)
        
    c_tags = set(cluster_tags[cluster_tags['n_cluster'] == c]['hashtags'].values[0].split(','))
    newtags = alltags - c_tags
    
    temp_result.append(tuple((c, newtags)))

In [20]:
extended_cluster_tags = pd.DataFrame(temp_result, columns=['n_cluster', 'hashtags'])

In [44]:
with open('output/2_step_tags_clusters.txt', 'w') as outfile:
    outfile.write('n_cluster\tdim\thashtags\n')
    for c in extended_cluster_tags['n_cluster']:
        cluster = extended_cluster_tags[extended_cluster_tags['n_cluster'] == c]['hashtags'].values[0]
        dim = len(cluster)
    
        outfile.write('{}\t{}\t{}\n'.format(c, dim, ','.join(list(cluster))))

In [26]:
#extended_cluster_tags['hashtags'] = extended_cluster_tags.apply(lambda x: x['hashtags'], for axis=1)
test = set()
for s in extended_cluster_tags['hashtags']:
    print len(s)
    test = test.union(s)

201
191
202
11
120
29
429
242
40
66
18
157
64
122
177
160
19
65
20
17
146
36
7
64
56
99
69
19


In [27]:
len(test)

2053

In [31]:
i = 10
print extended_cluster_tags.iloc[i]['n_cluster']
print extended_cluster_tags.iloc[i]['hashtags']

13
set(['loveher', 'libertyoflondon', 'palmsprings', 'angel', 'marcjacobs', 'pranzo', 'thegoodlife', 'womensrights', 'tompetty', 'whywetri', 'doit', 'somewhere', 'algarve', 'noplacelikehome', 'ladieswholunch', 'getfit', 'lovehim', 'epic'])


In [34]:
# compute list of tags for each user
usedtags = {}

it = network.BegNI()
V = network.GetNodes()
for i in range(V):
    nid = it.GetId()
    type = network.GetStrAttrDatN(nid, 'type')
    username = network.GetStrAttrDatN(nid, 'content')

    if type == 'user':
        tagsV = snap.TIntV()
        snap.GetNodesAtHop(network, nid, 2, tagsV, True)
        
        taglist = []
        for tid in tagsV:
            tagname = network.GetStrAttrDatN(tid, 'content')
            taglist.append(tagname)
            
        usedtags[username] = set(taglist)
    it.Next()

In [37]:
# store for next iterations/runs
pickle.dump(usedtags, open('output/user-tags-list.pkl', 'w'))

In [39]:
# compute participation 
tempResult = []
for u in usedtags:
    u_vector = usedtags[u]
    
    if len(u_vector)>0:
        u_result = [u]
        for c in extended_cluster_tags['n_cluster']:
            c_vector = extended_cluster_tags[extended_cluster_tags['n_cluster'] == c]['hashtags'].values[0]
            participation = float(len(u_vector.intersection(c_vector)))/len(u_vector)
            u_result.append(participation)
    else:
        u_result = [u]+[0 for i in extended_cluster_tags['n_cluster']]
    tempResult.append(tuple(u_result))

In [40]:
result_header = ['username']+[c for c in extended_cluster_tags['n_cluster']]
participationTable = pd.DataFrame(tempResult, columns=result_header)

In [41]:
participationTable.head()

,username,1,2,3,5,6,7,8,10,11,...,24,25,26,28,31,32,33,34,35,36
0,martaferrari9008,0.500000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0
1,rsersale,0.058824,0.0,0.000000,0.0,0.000000,0.000000,0.117647,0.000000,0.000000,...,0.0,0.0,0.058824,0.0,0.000000,0.0,0.176471,0.000000,0.000000,0.0
2,tlashab,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.111111,...,0.0,0.0,0.222222,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0
3,charr114,0.058824,0.0,0.117647,0.0,0.117647,0.058824,0.294118,0.235294,0.058824,...,0.0,0.0,0.000000,0.0,0.058824,0.0,0.000000,0.058824,0.117647,0.0
4,catespin,0.058824,0.0,0.000000,0.0,0.000000,0.000000,0.176471,0.058824,0.000000,...,0.0,0.0,0.000000,0.0,0.058824,0.0,0.000000,0.000000,0.058824,0.0


In [42]:
participationTable.to_csv('output/2_step_user_participation.csv', index=None)

In [60]:
first_step_cons = set()
for userlist in cluster_data['relevant_users']:
    for u in userlist:
        first_step_cons.add(u[0])

In [61]:
len(first_step_cons)

167

In [68]:
with open('output/2_step_user_groups.csv', 'w') as outfile:
    outfile.write('id_cluster\tnum_users\tlist_users\n')
    for i in extended_cluster_tags['n_cluster']:
        group = participationTable[participationTable[i] != 0.0]
        group.sort_values(by=i, ascending = False, inplace=True)
        
        userlist = [tuple((x['username'], x[i])) for index, x in group.iterrows() if x['username'] not in first_step_cons]
        
        dim = len(userlist)
        outfile.write('{}\t{}\t{}\n'.format(i, dim, str(userlist).strip('[]')))

/home/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
